In [1]:
import os, json, requests

from PIL import Image, ImageDraw, ImageFont, ImageOps
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO

In [ ]:
def invert_logo(path):
    logo = Image.open(path).convert("RGBA")
    r, g, b, a = logo.split()
    rgb_inverted = ImageOps.invert(Image.merge("RGB", (r, g, b)))
    return Image.merge("RGBA", (*rgb_inverted.split(), a))

def resize_logo(logo, percentage):
    new_width = int(image_size[0] * percentage)
    new_height = int(logo.height * new_width / logo.width)
    return logo.resize((new_width, new_height), Image.LANCZOS)

In [ ]:
index = 4
données = {}
dossier = ["../images/items/corrigés/", "../images/charmes/originaux/", "../images/screenshots/hollow_knight_areas", 
    "../images/screenshots/hollow_knight_boss/", "../images/screenshots/hollow_knight_npc"][index]
fichiers = os.listdir(dossier)

for fichier in fichiers:
    données[fichier] = {"x": 100, "y": 100}
    
print(json.dumps(données, indent=4, ensure_ascii=False))

In [ ]:
index = 3
dossier_source = ["../images/items/corrigés/", "../images/charmes/corrigés/", "../images/boss/corrigés/", "../images/npc/corrigés/"][index]
dossier_destination = ["../images/screenshots/hollow_knight_items/", "../images/screenshots/hollow_knight_charms/", "../images/screenshots/hollow_knight_boss", "../images/screenshots/hollow_knight_npc"][index]
marge = [0, 70, 25, 50][index]

couleur_fond = (127, 127, 127)

for fichier in os.listdir(dossier_source):
    if fichier.endswith(".webp"):
        chemin_fichier = os.path.join(dossier_source, fichier)
        image = Image.open(chemin_fichier).convert("RGBA")

        max_dimension = max(image.width, image.height) + 2 * marge
        nouvelle_taille = (max_dimension, max_dimension)

        image_avec_marge = Image.new("RGBA", nouvelle_taille, (0, 0, 0, 0))

        x_offset = (max_dimension - image.width) // 2
        y_offset = (max_dimension - image.height) // 2

        image_avec_marge.paste(image, (x_offset, y_offset))

        fond = Image.new("RGBA", nouvelle_taille, couleur_fond + (255,))
        image_finale = Image.alpha_composite(fond, image_avec_marge)

        image_finale.save(os.path.join(dossier_destination, fichier), "WEBP")

In [ ]:
zones = [
    "The Abyss", "Ancient Basin", "City of Tears", "Colosseum of Fools",
    "Crystal Peak", "Deepnest", "Dirtmouth", "Fog Canyon",
    "Forgotten Crossroads", "Fungal Wastes", "Godhome", "Greenpath",
    "The Hive", "Howling Cliffs", "Kingdom's Edge", "Queen's Gardens",
    "Resting Grounds", "Royal Waterways", "White Palace"
]

image_size = (500, 500)
font = ImageFont.truetype("../polices/Trajan.ttf", 50)
line_spacing = 20
margin = 20

top_logo = invert_logo("../images/ui/1_haut.webp")
bottom_logo = invert_logo("../images/ui/1_bas.webp")

top_logo = resize_logo(top_logo, 0.6)
bottom_logo = resize_logo(bottom_logo, 0.5)

for zone in zones:
    image = Image.new("RGB", image_size, (240, 240, 240))
    draw = ImageDraw.Draw(image)

    lines = zone.split(" ")
    text_width = max(draw.textbbox((0, 0), line, font=font)[2] for line in lines)
    text_height = sum(draw.textbbox((0, 0), line, font=font)[3] + line_spacing for line in lines) - line_spacing

    y_text = (image_size[1] - text_height) // 2

    logo_width, logo_height = top_logo.size
    y_logo_top = y_text - logo_height - margin
    x_logo_top = (image_size[0] - logo_width) // 2
    image.paste(top_logo, (x_logo_top, y_logo_top), top_logo)

    logo_width, logo_height = bottom_logo.size
    y_logo_bottom = y_text + text_height + margin + 10
    x_logo_bottom = (image_size[0] - logo_width) // 2
    image.paste(bottom_logo, (x_logo_bottom, y_logo_bottom), bottom_logo)

    for line in lines:
        bbox = draw.textbbox((0, 0), line, font=font)
        x_text = (image_size[0] - bbox[2]) // 2
        draw.text((x_text, y_text), line, fill="black", font=font)
        y_text += bbox[3] + line_spacing

    image.save(f"../images/screenshots/hollow_knight_areas/" + zone.lower().replace(" ", "_").replace("'", "_") + ".webp")

In [ ]:
index = 1
url = ["https://hollowknight.wiki/w/Category:Bosses_(Hollow_Knight)", "https://hollowknight.wiki/w/Category:NPCs_(Hollow_Knight)"][index]
path = ["boss", "npc"][index]
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
links = soup.select('table[align="center"] tbody tr td>a')

for link in links:
    href = link.get('href')
    text = link.get_text(strip=True)
    print(text)
    
    boss_page = requests.get(f"https://hollowknight.wiki{href}")
    boss_soup = BeautifulSoup(boss_page.content, 'html.parser')
    if text != "Dreamers":
        src = boss_soup.select_one('img.pi-image-thumbnail').get("src")
    else:
        src = boss_soup.select_one('img.mw-file-element[width="200"]').get("src")
    
    response = requests.get("https:" + src)
    image = Image.open(BytesIO(response.content))
    
    image_path = f"../images/{path}/originaux/" + text.replace(" ", "_").replace("'", "_").replace("&", "and").lower() + ".png"
    image.save(image_path)

In [8]:
index = 4
zone = ["hollow_knight_npc", "hollow_knight_items", "hollow_knight_areas", "hollow_knight_boss", "hollow_knight_charms"][index]

carte = Image.open(f"../images/cartes/{zone}/corrigée_petite.webp")

with open(f"../données/{zone}/coordonnées.json", "r") as file:
    coordonnées = json.load(file)
    
for image_name, coord in coordonnées.items():
    screenshot = Image.open(f"../images/screenshots/{zone}/{image_name}")
    
    if screenshot.mode != 'RGBA':
        screenshot = screenshot.convert('RGBA')
        
    x_length = 75
    y_length = 75
    screenshot = screenshot.resize((int(x_length), int(y_length)))
    
    x = coord["x"] - int(x_length / 2)
    y = coord["y"] - int(y_length / 2)
    
    carte.paste(screenshot, (x, y), screenshot)
carte.show()